In [45]:
import os
import sys
sys.path.append('../../src')
from hydra import initialize, compose
import pathlib
import pandas as pd
import numpy as np
import pickle

# from datasets.jobdataset import generate_dataset, _hfd5_from_dataframe
from datasets.job_hdf5 import hdf5_from_dataframe, get_career
import data_process.neg_sample as ng_sample
from utils.constants import DEFAULT_USER_COL,DEFAULT_ITEM_COL,DEFAULT_RATING_COL, DEFAULT_PREDICTION_COL
from implicit_eval import microsoft_eval,model_infer_df
# from implicit.als import AlternatingLeastSquares
# from implicit.bpr import BayesianPersonalizedRanking
# from implicit.lmf import LogisticMatrixFactorization
from implicit_build import bpr
from metrics import ranking

with initialize(version_base=None, config_path="../conf"):
    cfg = compose(config_name="config", overrides=[])

# Create the the data pipe

In [2]:
df_train_pos  = ng_sample.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_pos))

In [3]:
df_train_pos[DEFAULT_RATING_COL] = 1

In [4]:
df_train_pos[DEFAULT_USER_COL] = df_train_pos[DEFAULT_USER_COL].astype(str)
df_train_pos[DEFAULT_ITEM_COL] = df_train_pos[DEFAULT_ITEM_COL].astype(str)
df_train_pos[DEFAULT_USER_COL] = df_train_pos[DEFAULT_USER_COL].astype("category")
df_train_pos[DEFAULT_ITEM_COL] = df_train_pos[DEFAULT_ITEM_COL].astype("category")

In [5]:
# hdf5_from_dataframe(df_train_pos, pathlib.Path(cfg.path.root, cfg.file.hdf5))

In [6]:
# df_train_pos[DEFAULT_ITEM_COL].unique()

## Read data & Train

In [2]:
jobsid, usersid, user_job_app = get_career(pathlib.Path(cfg.path.root, cfg.file.hdf5))
model_path = "./models"

In [3]:
# user_job_app

In [4]:
# user_job_app = job_user_app.T.tocsr()
# bpr(model_path, user_job_app)
# als(model_path, user_job_app)
# lmf(model_path, user_job_app)

In [5]:
# df_train_pos.userid.unique()

In [7]:
def read_train_gd_csv(data_testgd_path, usecols):
    test_gddf = pd.read_csv(data_testgd_path, usecols=usecols)
    test_gddf[DEFAULT_USER_COL] = test_gddf[DEFAULT_USER_COL].astype('str')
    test_gddf[DEFAULT_ITEM_COL] = test_gddf[DEFAULT_ITEM_COL].astype('str')
    return test_gddf

In [8]:
df_test_ori = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.test))


In [9]:
df_train =read_train_gd_csv('../../data/jobs/leave_one_train_neg.csv', usecols=[DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL])

In [12]:
df_test =read_train_gd_csv('../../data/jobs/leave_one_test.csv', usecols=[DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL])


In [13]:
with open('./models/model_bpr.sav','rb') as pickle_in:
    model = pickle.load(pickle_in)
#     microsoft_eval(model, user_job_app, test_gddf=df_test, usersid=usersid, jobsid=jobsid, k=10, logger=None)
    
#     model.recommend(test_user_indexes, df_train[test_user_indexes], N=10)

In [14]:
df_test_ori[DEFAULT_USER_COL] = df_test_ori[DEFAULT_USER_COL].astype('str')
df_test_ori[DEFAULT_ITEM_COL] = df_test_ori[DEFAULT_ITEM_COL].astype('str')

In [34]:
apps_true = df_test_ori[df_test_ori['userid'].isin(['7', '1472090'])]

In [35]:
test_items = apps_true.itemid.values
test_items_indexes = np.searchsorted(jobsid, test_items)

In [36]:
test_users = apps_true[DEFAULT_USER_COL].unique()
test_user_indexes = np.searchsorted(usersid, test_users)
ids, scores = model.recommend(test_user_indexes, user_job_app[test_user_indexes],
                              N=10, filter_already_liked_items=True,
                             items=test_items_indexes)

In [13]:
predict_df = model_infer_df(df_test, usersid, jobsid, model, user_job_app, k=10)

In [46]:
def model_infer_df2(apps_true, usersid, jobsid, model, train, k):
#     reco_jobsid = []
    test_items = apps_true.itemid.values
    test_items_indexes = np.searchsorted(jobsid, test_items)
    
    test_users = apps_true[DEFAULT_USER_COL].unique()
    test_user_indexes = np.searchsorted(usersid, test_users)
    ids, scores = model.recommend(test_user_indexes, user_job_app[test_user_indexes],
                                  N=k, filter_already_liked_items=True,
                                 items=test_items_indexes)
    
    reco_jobsid = [jobsid[i] for i in ids.flatten()]
    reco_users = np.repeat(test_users, k)
    results = pd.DataFrame()
    results[DEFAULT_USER_COL] = reco_users
    results[DEFAULT_ITEM_COL] = reco_jobsid
    results[DEFAULT_RATING_COL] = 1
    results[DEFAULT_PREDICTION_COL] = scores.flatten()

    results[DEFAULT_USER_COL] = results[DEFAULT_USER_COL].astype('str')
    results[DEFAULT_ITEM_COL] = results[DEFAULT_ITEM_COL].astype('str')
    # results.to_csv('../data/clean/sub/predict_result.csv')
    return results

In [47]:
predict_df = model_infer_df2(apps_true, usersid, jobsid, model, user_job_app, k=10)

In [50]:
predict_df.shape

(20, 4)

In [51]:
df_test_ori

,userid,itemid,rating
0,7,703889,1
1,7,516136,0
2,7,544291,0
3,7,372900,0
4,7,954885,0
...,...,...,...
7483186,1472090,493793,0
7483187,1472090,1076061,0
7483188,1472090,807312,0
7483189,1472090,273190,0


In [19]:
# implicit.evaluation.AUC_at_k(model, )

In [44]:
k =3

In [73]:
gt_pos = np.asarray([0, 1, 0, 0, 0]) 
item_score = np.asarray([.5, .2, .3, .4, .6])
item_rank = item_score.argsort()[::-1]
item_rank

array([4, 0, 3, 2, 1])

In [70]:
pd_rank = item_rank

In [71]:
# gt_pos = np.asarray([0, 0, 0, 1, 1])  # [1, 0, 1]
# pd_rank = np.asarray([4, 3, 0,1,2])  # [1, 1, 1]

In [72]:
if k > 0:
    truncated_pd_rank = pd_rank[:k]
else:
    truncated_pd_rank = pd_rank
    
print(truncated_pd_rank)
pred = np.zeros_like(gt_pos)
pred[truncated_pd_rank] = 1
print('pred', pred)
print('choose the topk', pred*gt_pos)
tp = np.sum(pred * gt_pos)
print('tp', tp)
tp_fn = np.sum(gt_pos)
print('tp_fn', tp_fn)
tp_fp = np.sum(pred)
print('tp_fp', tp_fp)

[4 0 3]
pred [1 0 0 1 1]
choose the topk [0 0 0 0 0]
tp 0
tp_fn 1
tp_fp 3


In [37]:
score = np.asarray([0.3, 0.4, 0.1])

In [24]:
rank = score.argsort()[::-1]

In [25]:
rank

array([1, 0, 2])